In [111]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [113]:
df = pd.read_csv("tabla_comentarios.csv")
df  = df[~df.comentario.isnull()]
#df[np.isnan(df['sentimiento'])]
df['calidad'] = df['calidad_del_reviewer'].str.extract(r'(\d+)').replace(np.nan, 0)
df = df.drop(labels=['sentimiento', 'ciudad', 'pais','fecha_del_comentario', 'hora', "nombre_reviwer", "tiempo", "fecha_de_extraccion", "calidad_del_reviewer", "id_local", 'Unnamed: 0'], axis=1)
df['calificacion'] = np.where( df['calificacion'].str.extract(r'(\w)').apply(pd.to_numeric) > 3, 1 ,0)
df

,calificacion,comentario,calidad
0,1,Un sitio para tomar y una excelente cena. Su h...,26
1,1,"Es primer vez que consumo en este sitio , pero...",42
2,1,"Me gustó mucho el ambiente de este lugar, muy ...",42
3,1,"El lugar es muy agradable, tiene un amplio men...",491
4,1,La comida es muy rica. Tienen mesas al aire li...,97
...,...,...,...
505,0,(Traducción de Google) La música es muy buena....,11
506,1,(Traducción de Google) Tuve una buena experien...,40
507,1,(Traducción de Google) ¡¡¡¡La mejor música!!!!...,3
508,1,(Traducción de Google) Buena música\n\n(Origin...,50


In [114]:
X_train, X_test, y_train, y_test = train_test_split(df['comentario'], 
                                                    df['calificacion'], 
                                                    random_state=0)


In [115]:
print('X_train first entry:\n\n', X_train.iloc[1])

X_train first entry:

 Un sitio que se debe visitar al pasar por Medellín, un tinto, una cerveza y buena música.


In [117]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training data
vect = CountVectorizer().fit(X_train)

In [119]:
vect.get_feature_names()[::2000]

['000']

In [120]:
X_train_vectorized = vect.transform(X_train)

In [121]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression()

In [122]:
from sklearn.metrics import roc_auc_score

# Predict the transformed test documents
predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.553409090909091


In [123]:
# get the feature names as numpy array
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['pero' 'no' 'tan' 'mala' 'sad' 'ubicación' 'algo' 'precio' 'mal' 'pues']

Largest Coefs: 
['agradable' 'excelente' 'música' 'delicioso' 'lugar' 'mejor' 'buena'
 'super' 'amigos' 'deliciosa']


In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

160

In [125]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.5


In [126]:
feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['mesa' 'cuando' 'quieres' 'mucho' 'fui' 'mesas' 'también' 'precio'
 'puedes' 'está']

Largest tfidf: 
['atención' 'delicioso' 'deliciosos' 'es' 'una' 'excelente' 'lugar' 'pues'
 'rico' 'argentina']


In [127]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['no' 'pero' 'que' 'pues' 'lo' 'el' 'tan' 'mal' 'precio' 'nada']

Largest Coefs: 
['excelente' 'lugar' 'buena' 'música' 'agradable' 'delicioso' 'mejor'
 'muy' 'deliciosa' 'amigos']


In [128]:
vect = CountVectorizer(min_df=7, ngram_range=(1,4)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())

175

In [129]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.6738636363636363


In [130]:
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['pero' 'precio' 'no' 'algo' 'servicio' 'un bar' 'lo' 'un lugar' 'fue'
 'el']

Largest Coefs: 
['agradable' 'de los' 'excelente' 'delicioso' 'la música' 'deliciosos'
 'música' 'amigos' 'por' 'deliciosa']


In [131]:
vect.get_feature_names()[::2000]

['acogedor']